<a href="https://colab.research.google.com/github/Olfeng-xalaz/Fallstudie_SCM/blob/Test1/Fallstudie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pip als Paketmanager
! pip install -q pyscipopt
! pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 50.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import math
from pyscipopt import Model, quicksum

# Optimierungsmodell zur Elektrifizierung der Logistik

### Import der CSV Dateien

In [3]:
import os

# Prüfen ob in Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    # In Colab: Repository klonen (nur einmal)
    if not os.path.exists('Fallstudie_SCM'):
        !git clone https://github.com/Olfeng-xalaz/Fallstudie_SCM.git
    folder = "Fallstudie_SCM/DataCSV"
else:
    # Lokal in VS Code: Relativer Pfad
    folder = os.path.join(os.path.dirname(__file__), "DataCSV") if "__file__" in dir() else "DataCSV"

Cloning into 'Fallstudie_SCM'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 65 (delta 21), reused 13 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (65/65), 27.92 KiB | 5.58 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [4]:
chargers = pd.read_csv(f"{folder}/chargers.csv", sep=";")

In [5]:
chargers.head()

,charger_model,capex_yearly,opex_yearly,max_power,charging_spots
0,Alpitronic-50,3000,1000,50,2
1,Alpitronic-200,10000,1500,200,2
2,Alpitronic-400,16000,2000,400,2


In [6]:
dtrucks_specs = pd.read_csv(f"{folder}/diesel_trucks.csv", sep=";")

In [7]:
dtrucks_specs.head()

,truck_model,capex_yearly,opex_yearly,avg_diesel_per_100km,kfz_yearly,gross_vehicle_weight,emission_class,co2_emission_class
0,ActrosL,24000,6000,26,556,40,EURO 6,1


In [8]:
etrucks_specs = pd.read_csv(f"{folder}/electric_trucks.csv", sep=";")

In [9]:
etrucks_specs.head()

,truck_model,capex_yearly,opex_yearly,avg_energy_kWh_per_100km,thg_yearly,max_power,soc_max_kWh
0,eActros600,60000,6000,110,1000,400,621
1,eActros400,50000,5000,105,1000,400,414


In [10]:
routes = pd.read_csv(f"{folder}/routes.csv", sep=";")

In [11]:
routes

,route_id,route_name,distance_total,distance_toll,starttime,endtime
0,t-4,Nahverkehr,250,150,06:45,17:15
1,t-5,Nahverkehr,250,150,06:30,17:00
2,t-6,Nahverkehr,250,150,06:00,16:30
3,s-1,Ditzingen,120,32,05:30,15:30
4,s-2,Ditzingen,120,32,06:00,16:00
5,s-3,Ditzingen,120,32,09:00,16:00
6,s-4,Ditzingen,120,32,06:30,16:30
7,w1,Ditzingen,100,32,05:30,15:30
8,w2,Ditzingen,100,32,08:00,18:00
9,w3,Ditzingen,100,32,06:45,16:45


### Indexmengen

In [12]:
#Erstellen einer Modellinstanz
scip = Model()

In [13]:
R = routes["route_id"].unique() # Menge der Routen
C = chargers["charger_model"].unique() # Menge der Charger-Modelle
L = pd.concat([dtrucks_specs["truck_model"],etrucks_specs["truck_model"]]).unique() # Menge der Fahrzeugmodelle
P = range(1, 21) # Potenzielle Fahrzeuge
T = range(1, 97) # 96 Zeitintervalle pro Tag
I = range(0, 4) # Potenzielle Säulen-Slots
# T_night = [t for t in T if t > 18*4 or t <= 6*4] # Nachtintervalle: 18:00–06:00


print("R (Routen):", R)
print("C (Charger):", C)
print("L (Fahrzeugmodelle):", L)
print("P (potenzielle Fahrzeuge):", list(P))
print("T (Zeitintervalle):", list(T))
print("I (Säulen):", list(I))
# print("T_night (Nachtintervalle):", list(T_night))


R (Routen): ['t-4' 't-5' 't-6' 's-1' 's-2' 's-3' 's-4' 'w1' 'w2' 'w3' 'w4' 'w5' 'w6'
 'w7' 'r1' 'r2' 'r3' 'h3' 'h4' 'k1']
C (Charger): ['Alpitronic-50' 'Alpitronic-200' 'Alpitronic-400']
L (Fahrzeugmodelle): ['ActrosL' 'eActros600' 'eActros400']
P (potenzielle Fahrzeuge): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
T (Zeitintervalle): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96]
I (Säulen): [0, 1, 2, 3]


### Parameter

In [14]:
# 3) Parameter aus routes.csv
# -----------------------------
dist_total = dict(zip(routes["route_id"], routes["distance_total"]))
dist_toll  = dict(zip(routes["route_id"], routes["distance_toll"]))
start_time = dict(zip(routes["route_id"], routes["starttime"]))
end_time   = dict(zip(routes["route_id"], routes["endtime"]))

# Hinweis:
# A[r,t] (binär: Tour r läuft im Intervall t) und g[r,t] (kWh-Verbrauch pro Intervall)
# hängen von Zeitdiskretisierung + Tourdauer ab.
# Das bauen wir später, sobald klar ist, wie starttime/endtime formatiert sind.

# -----------------------------
# 4) Parameter aus chargers.csv
# -----------------------------
capex_ch = dict(zip(chargers["charger_model"], chargers["capex_yearly"]))
opex_ch  = dict(zip(chargers["charger_model"], chargers["opex_yearly"]))
pmax_ch  = dict(zip(chargers["charger_model"], chargers["max_power"]))        # kW
spots_ch = dict(zip(chargers["charger_model"], chargers["charging_spots"]))   # Anzahl Ladepunkte

# -----------------------------
# 5) Parameter aus electric_trucks.csv
# -----------------------------
capex_veh = {}
opex_veh  = {}

cons_e = {}       # kWh/100km
thg_e  = {}       # €/a
pmax_veh = {}     # kW max Ladeleistung
batt_kwh = {}     # kWh Batterie (soc_max)

for _, row in etrucks_specs.iterrows():
    m = row["truck_model"]
    capex_veh[m] = row["capex_yearly"]
    opex_veh[m]  = row["opex_yearly"]
    cons_e[m]    = row["avg_energy_kWh_per_100km"]
    thg_e[m]     = row["thg_yearly"]
    pmax_veh[m]  = row["max_power"]
    batt_kwh[m]  = row["soc_max_kWh"]

# -----------------------------
# 6) Parameter aus diesel_trucks.csv
# -----------------------------
kfz_d = {}       # €/a
cons_d = {}      # ggf. l/100km (nur falls du Diesel-Kraftstoffkosten modellierst)

for _, row in dtrucks_specs.iterrows():
    m = row["truck_model"]
    capex_veh[m] = row["capex_yearly"]
    opex_veh[m]  = row["opex_yearly"]
    if "kfz_yearly" in dtrucks_specs.columns:
        kfz_d[m] = row["kfz_yearly"]
    if "avg_diesel_per_100km" in dtrucks_specs.columns:
        cons_d[m] = row["avg_diesel_per_100km"]

# -----------------------------
# 7) Abgeleitete Parameter: Energiebedarf pro Route und e-Lkw (E[r,e])
# -----------------------------
E_route_e = {}  # (r,e) -> kWh

for r in R:
    for e in cons_e.keys():
        E_route_e[(r, e)] = dist_total[r] * cons_e[e] / 100.0

# -----------------------------
# 8) Falltext-Parameter (Konstanten)
# -----------------------------
N_days = 260
delta_h = 0.25  # 15 Minuten = 0.25 Stunden
diesel = 1.60

# Netz & Tarif
P_grid_max = 500.0 # max kW am Depot
P_grid_add = 500.0 # Zusatzleistung bei Netzausbau
capex_grid_add = 10000.0 # Kosten/Jahr für Netzausbau

c_energy = 0.25   # Arbeitspreis Strom in €/kWh
c_capex = 1000.0   # Stromkosten in €/a
c_peak = 150.0    # Leistungspreis in €/kW

# Maut
c_toll = 0.34      # €/km mautpflichtig

# Speicher
c_capex_bat_kW = 30.0      # Batteriekosten in €/kW
c_capex_bat_kWh = 350.0     # Batteriekosten in €/kWh
c_opex_bat = 0.02*c_capex_bat_kW*c_capex_bat_kWh # hier muss noch mit der tatsächlichen kW und kWh Anzahl multipliziert werden!!!
roundtrip_eff = 0.98 # Round-Trip Efficiency aka Wirkungsgrad
eta = math.sqrt(roundtrip_eff)  # für lineare Lade/Entlade-Gleichungen
dod = 0.975 # Max. Entladetiefe
soc_bat_min_frac = 1.0 - dod    # = 0.025

# -----------------------------
# 9) Kurzer Test-Print der wichtigsten Parameter
# -----------------------------
print("Beispiel dist_total[r]:", list(dist_total.items())[:3]); print()
print("Charger pmax (kW):", pmax_ch); print()
print("e-Lkw batt_kwh:", batt_kwh); print()
print("Energiebedarf Beispiel (erste Route, e400/e600):")
first_r = R[0]
for e in cons_e.keys():
    print(" ", (first_r, e), "=", E_route_e[(first_r, e)], "kWh")
print()

print("Konstanten: P_grid_max =", P_grid_max, "| c_energy =", c_energy, "| c_toll =", c_toll); print()
print("Speicher: eta =", eta, "| soc_bat_min_frac =", soc_bat_min_frac); print()

Beispiel dist_total[r]: [('t-4', 250), ('t-5', 250), ('t-6', 250)]

Charger pmax (kW): {'Alpitronic-50': 50, 'Alpitronic-200': 200, 'Alpitronic-400': 400}

e-Lkw batt_kwh: {'eActros600': 621, 'eActros400': 414}

Energiebedarf Beispiel (erste Route, e400/e600):
  ('t-4', 'eActros600') = 275.0 kWh
  ('t-4', 'eActros400') = 262.5 kWh

Konstanten: P_grid_max = 500.0 | c_energy = 0.25 | c_toll = 0.34

Speicher: eta = 0.9899494936611666 | soc_bat_min_frac = 0.025000000000000022



### Entscheidungsvariablen

In [15]:
# ============================================================
# Entscheidungsvariablen (PySCIPOpt) – übersichtlich gegliedert
# ============================================================
# Voraussetzung: Indexmengen sind bereits definiert:
# R (Routen), T (Zeitintervalle), P (potenzielle Fahrzeuge),
# I (Säulen-Slots), C (Charger-Modelle), L (Fahrzeugmodelle)

from pyscipopt import Model

model = Model("Fleet_Charging_Optimization")

# ------------------------------------------------------------
# 1) STRATEGISCHE VARIABLEN
# ------------------------------------------------------------

# 1.1 Fahrzeugtyp je potenziellem Fahrzeug k
# u[k,l] = 1, wenn Fahrzeug k als Modell l betrieben wird
u = {}
for k in P:
    for l in L:
        u[k, l] = model.addVar(vtype="B", name=f"u_{k}_{l}")

# 1.2 Ladesäulen-Auswahl je Slot i (max. 3 Slots)
# z[i,c] = 1, wenn Säule i als Charger-Modell c installiert wird
z = {}
for i in I:
    for c in C:
        z[i, c] = model.addVar(vtype="B", name=f"z_{i}_{c}")

# 1.3 Netzanschluss-Erweiterung (ja/nein)
y_ext = model.addVar(vtype="B", name="y_ext")

# 1.4 Batteriespeicher-Design: Leistung und Kapazität
P_bat = model.addVar(lb=0, vtype="C", name="P_bat")  # kW
E_bat = model.addVar(lb=0, vtype="C", name="E_bat")  # kWh


# ------------------------------------------------------------
# 2) TAKTISCHE VARIABLEN
# ------------------------------------------------------------

# 2.1 Tourzuordnung
# x[r,k] = 1, wenn Route r von Fahrzeug k gefahren wird
x = {}
for r in R:
    for k in P:
        x[r, k] = model.addVar(vtype="B", name=f"x_{r}_{k}")


# ------------------------------------------------------------
# 3) OPERATIVE VARIABLEN (LADEMANAGEMENT)
# ------------------------------------------------------------

# 3.1 Belegung eines Ladepunkts an Säule i
# b[k,i,t] = 1, wenn Fahrzeug k in t einen Ladepunkt an Säule i belegt
b = {}
for k in P:
    for i in I:
        for t in T:
            b[k, i, t] = model.addVar(vtype="B", name=f"b_{k}_{i}_{t}")

# 3.2 Ladeleistung Fahrzeug -> Säule
# p_charge[k,i,t] = Ladeleistung (kW) von k an i in t
p_charge = {}
for k in P:
    for i in I:
        for t in T:
            p_charge[k, i, t] = model.addVar(lb=0, vtype="C", name=f"p_{k}_{i}_{t}")

# 3.3 SOC der Fahrzeugbatterie
# SOC[k,t] = Ladezustand (kWh) von Fahrzeug k in t
SOC = {}
for k in P:
    for t in T:
        SOC[k, t] = model.addVar(lb=0, vtype="C", name=f"SOC_{k}_{t}")


# ------------------------------------------------------------
# 4) NETZVARIABLEN (NETZLAST & PEAK)
# ------------------------------------------------------------

# 4.1 Netzbezugsleistung je Zeitintervall
# P_grid[t] = Netzleistung (kW) in t
P_grid = {}
for t in T:
    P_grid[t] = model.addVar(lb=0, vtype="C", name=f"P_grid_{t}")

# 4.2 Jahres-Peak (für Leistungspreis)
P_peak = model.addVar(lb=0, vtype="C", name="P_peak")


# ------------------------------------------------------------
# 5) SPEICHER-BETRIEBSVARIABLEN
# ------------------------------------------------------------

# p_bat_ch[t]  = Speicher lädt (kW)
# p_bat_dis[t] = Speicher entlädt (kW)
# SOC_bat[t]   = Speicher-SOC (kWh)
p_bat_ch = {}
p_bat_dis = {}
SOC_bat = {}

for t in T:
    p_bat_ch[t]  = model.addVar(lb=0, vtype="C", name=f"p_bat_ch_{t}")
    p_bat_dis[t] = model.addVar(lb=0, vtype="C", name=f"p_bat_dis_{t}")
    SOC_bat[t]   = model.addVar(lb=0, vtype="C", name=f"SOC_bat_{t}")


# ------------------------------------------------------------
# 6) OPTIONAL: HILFSVARIABLEN (z.B. Diesel-Maut linear)
# ------------------------------------------------------------

# w[r,k] = 1, wenn Route r von einem Diesel-Fahrzeug k gefahren wird
# (nützlich, um Mautkosten ohne Produkt x[r,k]*u[k,Diesel] zu modellieren)
w = {}
for r in R:
    for k in P:
        w[r, k] = model.addVar(vtype="B", name=f"w_{r}_{k}")


# ------------------------------------------------------------
# 7) DEBUG / CHECK
# ------------------------------------------------------------
model.hideOutput(True)  # optional: Solver-Ausgabe unterdrücken
print("✅ Variablen angelegt.")
print("Anzahl Variablen:", model.getNVars())
print()


✅ Variablen angelegt.
Anzahl Variablen: 18540



### Zielfunktion

### Restriktionen

### Berechnung des Ergebnisses